# Các Agent AutoGen trong Sản Xuất: Quan Sát & Đánh Giá

Trong hướng dẫn này, chúng ta sẽ tìm hiểu cách **giám sát các bước nội bộ (traces) của [các agent Autogen](https://github.com/microsoft/autogen)** và **đánh giá hiệu suất của chúng** bằng cách sử dụng [Langfuse](https://langfuse.com).

Hướng dẫn này bao gồm các chỉ số đánh giá **trực tuyến** và **ngoại tuyến** mà các nhóm sử dụng để đưa các agent vào sản xuất một cách nhanh chóng và đáng tin cậy.

**Tại sao việc Đánh Giá agent AI lại quan trọng:**
- Gỡ lỗi các vấn đề khi nhiệm vụ thất bại hoặc tạo ra kết quả không tối ưu
- Giám sát chi phí và hiệu suất theo thời gian thực
- Cải thiện độ tin cậy và an toàn thông qua phản hồi liên tục


## Bước 1: Thiết lập các biến môi trường

Lấy khóa API của Langfuse bằng cách đăng ký [Langfuse Cloud](https://cloud.langfuse.com/) hoặc [tự triển khai Langfuse](https://langfuse.com/self-hosting).

_**Lưu ý:** Người tự triển khai có thể sử dụng [các module Terraform](https://langfuse.com/self-hosting/azure) để triển khai Langfuse trên Azure. Ngoài ra, bạn có thể triển khai Langfuse trên Kubernetes bằng [Helm chart](https://langfuse.com/self-hosting/kubernetes-helm)._


In [5]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

Với các biến môi trường được thiết lập, chúng ta có thể khởi tạo client Langfuse. `get_client()` khởi tạo client Langfuse bằng cách sử dụng thông tin xác thực được cung cấp trong các biến môi trường.


In [6]:
from langfuse import Langfuse
 
# Filter out Autogen OpenTelemetryspans
langfuse = Langfuse(
    blocked_instrumentation_scopes=["autogen SingleThreadedAgentRuntime"]
)
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## Bước 2: Khởi tạo Công cụ OpenLit

Bây giờ, chúng ta sẽ khởi tạo công cụ [OpenLit](https://github.com/openlit/openlit). OpenLit tự động ghi lại các thao tác AutoGen và xuất các span OpenTelemetry (OTel) sang Langfuse.


In [7]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True, disabled_instrumentors=["mistral"])

## Bước 3: Chạy tác nhân của bạn

Bây giờ chúng ta thiết lập một tác nhân nhiều lượt để kiểm tra công cụ của mình.


In [2]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": False
    },
)

In [8]:
# 🍴 Agent 1 – proposes ONE healthy meal idea each turn
meal_planner_agent = AssistantAgent(
    "meal_planner_agent",
    model_client=client,
    description="A seasoned meal-planning coach who suggests balanced meals.",
    system_message="""
    You are a Meal-Planning Assistant with a decade of experience helping busy people prepare meals.
    Goal: propose the single best meal (breakfast, lunch, or dinner) given the user's context.
    Each response must contain ONLY one complete meal idea (title + very brief component list) — no extras.
    Keep it concise: skip greetings, chit-chat, and filler.
    """,
)

# 🥗 Agent 2 – checks nutritional quality & variety
nutritionist_agent = AssistantAgent(
    "nutritionist_agent",
    model_client=client,
    description="A registered dietitian ensuring meals meet nutritional standards.",
    system_message="""
    You are a Nutritionist focused on whole-food, macro-balanced eating.
    Evaluate the meal_planner_agent’s recommendation.
    If the meal is nutritionally sound, sufficiently varied, and portion-appropriate, respond with 'APPROVE'.
    Otherwise, give high-level guidance on how to improve it (e.g. 'add a plant-based protein') — do NOT provide a full alternative recipe.
    """,
)

In [9]:
# ✅ Chat stops once the nutritionist says APPROVE
termination = TextMentionTermination("APPROVE")

# 🔄 Alternate turns between the two agents until termination
team = RoundRobinGroupChat(
    [meal_planner_agent, nutritionist_agent],
    termination_condition=termination,
)

# Example kickoff
user_input = "I'm looking for a quick, delicious dinner I can prep after work. I have 30 minutes and minimal clean-up is ideal."

In [ ]:
with langfuse.start_as_current_span(name="create_meal_plan") as span:
    async for message in team.run_stream(task=user_input):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

# Flush the trace to Langfuse for short-lived environments such as Jupyter Notebooks
langfuse.flush()

### Cấu trúc Trace

Langfuse ghi lại một **trace** chứa các **span**, đại diện cho từng bước trong logic của agent của bạn. Ở đây, trace bao gồm toàn bộ quá trình chạy của agent và các sub-span cho:
- Agent lập kế hoạch bữa ăn
- Các agent chuyên gia dinh dưỡng

Bạn có thể kiểm tra những điều này để thấy chính xác thời gian được sử dụng ở đâu, số lượng token được sử dụng, và các thông tin khác:

![Cây trace trong Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/trace-tree.png)

_[Liên kết đến trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


## Đánh Giá Trực Tuyến

Đánh giá trực tuyến đề cập đến việc đánh giá tác nhân trong môi trường thực tế, tức là trong quá trình sử dụng thực tế khi triển khai. Điều này bao gồm việc giám sát hiệu suất của tác nhân trên các tương tác thực tế với người dùng và phân tích kết quả liên tục.

### Các Chỉ Số Thường Theo Dõi Trong Sản Xuất

1. **Chi phí** — Công cụ đo lường sẽ ghi lại việc sử dụng token, bạn có thể chuyển đổi thành chi phí ước tính bằng cách gán giá cho mỗi token.
2. **Độ trễ** — Theo dõi thời gian hoàn thành mỗi bước hoặc toàn bộ quy trình.
3. **Phản hồi từ người dùng** — Người dùng có thể cung cấp phản hồi trực tiếp (thích hoặc không thích) để giúp cải thiện hoặc sửa lỗi của tác nhân.
4. **LLM-as-a-Judge** — Sử dụng một LLM riêng để đánh giá đầu ra của tác nhân gần như theo thời gian thực (ví dụ: kiểm tra tính độc hại hoặc độ chính xác).

Dưới đây là các ví dụ về những chỉ số này.


#### 1. Chi phí

Dưới đây là ảnh chụp màn hình hiển thị mức sử dụng cho các lần gọi `gpt-4o-mini`. Điều này hữu ích để xem các bước tốn kém và tối ưu hóa tác nhân của bạn.

![Chi phí](https://langfuse.com/images/cookbook/example-autogen-evaluation/gpt-4o-costs.png) 

_[Liên kết đến dấu vết](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


#### 2. Độ trễ

Chúng ta cũng có thể xem mất bao lâu để hoàn thành mỗi bước. Trong ví dụ dưới đây, toàn bộ quá trình chạy mất khoảng 3 giây, và bạn có thể phân tích theo từng bước. Điều này giúp bạn xác định các điểm nghẽn và tối ưu hóa tác nhân của mình.

![Độ trễ](https://langfuse.com/images/cookbook/example-autogen-evaluation/agent-latency.png) 

_[Liên kết đến dấu vết](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64?display=timeline)_


#### 3. Phản hồi từ người dùng

Nếu tác nhân của bạn được tích hợp vào giao diện người dùng, bạn có thể ghi lại phản hồi trực tiếp từ người dùng (như biểu tượng thích/không thích trong giao diện trò chuyện).


In [10]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="autogen-request-user-feedback-1") as span:
    
    async for message in team.run_stream(task="Create a meal with potatoes"):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was delicious, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:
    # ... Autogen execution ...

    async for message in team.run_stream(task="I am allergic to gluten."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )

id='da068880-22ae-4f01-9f01-2bb231939089' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 43, 732669, tzinfo=datetime.timezone.utc) content='Create a meal with potatoes' type='TextMessage'
id='ad937ce4-3534-493f-824b-ca9c226b5287' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=95, completion_tokens=30) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 186423, tzinfo=datetime.timezone.utc) content='Potato and Spinach Frittata  \n- Eggs  \n- Potatoes  \n- Fresh spinach  \n- Onion  \n- Cheese (optional)  ' type='TextMessage'
id='50fd33c1-057f-49fe-afad-ee86d164296d' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=132, completion_tokens=4) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 581059, tzinfo=datetime.timezone.utc) content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
id='e371de6c-e5fc-42c1-8eda-e5b8cd5accab' source='user' models_usage=None met

In [ ]:
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id="predefined_trace_id",
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

Phản hồi của người dùng sau đó được ghi lại trong Langfuse:

![Phản hồi của người dùng đang được ghi lại trong Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/user-feedback.png)


#### 4. Đánh giá tự động bằng LLM-as-a-Judge

LLM-as-a-Judge là một cách khác để tự động đánh giá đầu ra của tác nhân. Bạn có thể thiết lập một cuộc gọi LLM riêng để đánh giá tính chính xác, độc hại, phong cách, hoặc bất kỳ tiêu chí nào khác mà bạn quan tâm.

**Quy trình làm việc**:
1. Bạn định nghĩa một **Mẫu Đánh Giá**, ví dụ: "Kiểm tra xem văn bản có độc hại không."
2. Bạn thiết lập một mô hình được sử dụng làm judge-model; trong trường hợp này là `gpt-4o-mini` được truy vấn qua Azure.
3. Mỗi khi tác nhân của bạn tạo ra đầu ra, bạn chuyển đầu ra đó đến LLM "judge" cùng với mẫu đánh giá.
4. LLM "judge" phản hồi với một xếp hạng hoặc nhãn mà bạn ghi lại vào công cụ quan sát của mình.

Ví dụ từ Langfuse:

![LLM-as-a-Judge Evaluator](https://langfuse.com/images/cookbook/example-autogen-evaluation/evaluator.png)


In [12]:
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:

    async for message in team.run_stream(task="I am a picky eater and not sure if you find something for me."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message) 

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

langfuse.flush()

id='eefc628d-502f-451a-8f70-be486f62f8c5' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 29, 171393, tzinfo=datetime.timezone.utc) content='I am a picky eater and not sure if you find something for me.' type='TextMessage'
id='13b3e14b-bcf7-42a5-80d6-54b0c7be765e' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=352, completion_tokens=27) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 30, 433516, tzinfo=datetime.timezone.utc) content='Chicken Alfredo Pasta  \n- Gluten-free pasta  \n- Grilled chicken breast  \n- Heavy cream  \n- Parmesan cheese  \n- Garlic  ' type='TextMessage'
id='550f2dee-0e08-4bbd-b67f-991b467328f1' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=386, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 31, 505173, tzinfo=datetime.timezone.utc) content='Consider incorporating some vegetables, like spinach or broccoli, to increase the nutrien

Bạn có thể thấy rằng câu trả lời của ví dụ này được đánh giá là "không độc hại".

![LLM-as-a-Judge Evaluation Score](https://langfuse.com/images/cookbook/example-autogen-evaluation/llm-as-a-judge-score.png)


#### 5. Tổng Quan Về Các Chỉ Số Quan Sát

Tất cả các chỉ số này có thể được hiển thị cùng nhau trên các bảng điều khiển. Điều này cho phép bạn nhanh chóng thấy cách mà agent của bạn hoạt động qua nhiều phiên và giúp bạn theo dõi các chỉ số chất lượng theo thời gian.

![Tổng quan về các chỉ số quan sát](https://langfuse.com/images/cookbook/example-autogen-evaluation/dashboard.png)


## Đánh giá Ngoại tuyến

Đánh giá trực tuyến rất quan trọng để có phản hồi ngay lập tức, nhưng bạn cũng cần **đánh giá ngoại tuyến**—các kiểm tra có hệ thống trước hoặc trong quá trình phát triển. Điều này giúp duy trì chất lượng và độ tin cậy trước khi triển khai các thay đổi vào môi trường sản xuất.


### Đánh giá Bộ Dữ Liệu

Trong đánh giá ngoại tuyến, bạn thường:
1. Có một bộ dữ liệu chuẩn (bao gồm các cặp câu hỏi và câu trả lời mong đợi)
2. Chạy tác nhân của bạn trên bộ dữ liệu đó
3. So sánh kết quả đầu ra với kết quả mong đợi hoặc sử dụng một cơ chế chấm điểm bổ sung

Dưới đây, chúng tôi minh họa cách tiếp cận này với [q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset), bộ dữ liệu chứa các câu hỏi và câu trả lời mong đợi.


In [16]:
import pandas as pd
from datasets import load_dataset
 
# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

/Users/jannik/Documents/GitHub/ai-agents-for-beginners/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First few rows of search-dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers surrendered to t

Tiếp theo, chúng ta tạo một thực thể tập dữ liệu trong Langfuse để theo dõi các lần chạy. Sau đó, chúng ta thêm từng mục từ tập dữ liệu vào hệ thống.


In [17]:
from langfuse import Langfuse
langfuse = Langfuse()
 
langfuse_dataset_name = "qa-dataset_autogen-agent"
 
# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="q&a dataset uploaded from Hugging Face",
    metadata={
        "date": "2025-03-21",
        "type": "benchmark"
    }
)

Dataset(id='cmcm7524d00kjad07s2cjwqcf', name='qa-dataset_autogen-agent', description='q&a dataset uploaded from Hugging Face', metadata={'date': '2025-03-21', 'type': 'benchmark'}, project_id='cloramnkj0002jz088vzn1ja4', created_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc))

In [18]:
df_25 = df.sample(25) # For this example, we upload only 25 dataset questions

for idx, row in df_25.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )

![Các mục dữ liệu trong Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/example-dataset.png)


#### Chạy Agent trên Tập Dữ Liệu

Đầu tiên, chúng ta tạo một agent Autogen đơn giản để trả lời các câu hỏi bằng cách sử dụng các mô hình Azure OpenAI.


In [8]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    max_tokens=5000,
    model_info={
        "json_output": True,
        "function_calling": False,
        "vision": False,
        "family": "unknown",
        "structured_output": True,
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

In [18]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are participant in a quizz show and you are given a question. You need to create a short answer to the question.",
)

Sau đó, chúng ta định nghĩa một hàm trợ giúp `my_agent()`.


In [19]:
async def my_agent(user_query: str):

    with langfuse.start_as_current_span(name="autogen-trace") as span:

        # Execute the agent response
        response = await agent.on_messages(
            [TextMessage(content=user_query, source="user")],
            cancellation_token=CancellationToken(),
        )

        span.update_trace(
            input=user_query,
            output=response.chat_message.content,
        )

    return str(response.chat_message.content)

# Test the function
await my_agent("What is the capital of France?")

'The capital of France is Paris.'

Cuối cùng, chúng ta lặp qua từng mục trong tập dữ liệu, chạy tác nhân, và liên kết dấu vết với mục trong tập dữ liệu. Chúng ta cũng có thể đính kèm một điểm đánh giá nhanh nếu muốn.


In [20]:
dataset_name = "qa-dataset_autogen-agent"
current_run_name = "dev_tasks_run-autogen_gpt-4.1" # Identifies this specific evaluation run
current_run_metadata={"model_provider": "Azure", "model": "gpt-4.1"}
current_run_description="Evaluation run for Autogen model on July 3rd"

dataset = langfuse.get_dataset('qa-dataset_autogen-agent')

for item in dataset.items:
    print(f"Running evaluation for item: {item.id} (Input: {item.input})")
 
    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata=current_run_metadata,
        run_description=current_run_description
    ) as root_span: 
        # All subsequent langfuse operations within this block are part of this trace.
        generated_answer = await my_agent(user_query = item.input["text"])
    
    print("Generated Answer: ", generated_answer)
 
print(f"\nFinished processing dataset '{dataset_name}' for run '{current_run_name}'.")

langfuse.flush()

Running evaluation for item: 09810cc4-9992-4712-a3b2-7224da31776a (Input: {'text': 'In Hindu mythology, which deity is the Ganges river dolphin associated with?'})
Generated Answer:  In Hindu mythology, the Ganges river dolphin is associated with the deity Ganga.
Running evaluation for item: bb113f94-7723-47c6-8c34-59d883044514 (Input: {'text': 'What significant discovery did the LHCb collaboration report in 2015?'})
Generated Answer:  In 2015, the LHCb collaboration reported the discovery of pentaquark particles.
Running evaluation for item: 4d8ae54e-ceab-46d0-ad2c-6e8e223589a9 (Input: {'text': 'What is the MÄ\x81ori name for the red-crowned parakeet?'})
Generated Answer:  The Māori name for the red-crowned parakeet is kākāriki.
Running evaluation for item: 21e5a0d5-f619-4a73-868e-9955053b3e72 (Input: {'text': 'Who starred in the 1978 television film adaptation of Les MisÃ©rables?'})
Generated Answer:  Richard Jordan starred as Jean Valjean in the 1978 television film adaptation of Le

CÁC QUY TẮC QUAN TRỌNG:  
1. KHÔNG thêm '''markdown hoặc bất kỳ thẻ nào khác xung quanh bản dịch  
2. Đảm bảo bản dịch không quá sát nghĩa từng từ  
3. Dịch cả các chú thích  
4. Tệp này được viết theo định dạng Markdown - không xử lý nó như XML hoặc HTML  
5. Không dịch:  
   - [!NOTE], [!WARNING], [!TIP], [!IMPORTANT], [!CAUTION]  
   - Tên biến, tên hàm, tên lớp  
   - Các placeholder như @@INLINE_CODE_x@@ hoặc @@CODE_BLOCK_x@@  
   - URL hoặc đường dẫn  
6. Giữ nguyên toàn bộ định dạng Markdown gốc  
7. Chỉ trả về nội dung đã dịch mà không thêm bất kỳ thẻ hoặc đánh dấu nào khác  

Bạn có thể lặp lại quy trình này với các cấu hình tác nhân khác nhau như:  
- Các mô hình (gpt-4o-mini, gpt-4.1, v.v.)  
- Các lời nhắc (prompts)  
- Công cụ (tìm kiếm so với không tìm kiếm)  
- Độ phức tạp của tác nhân (đa tác nhân so với đơn tác nhân)  

Sau đó so sánh chúng cạnh nhau trong Langfuse. Trong ví dụ này, tôi đã chạy tác nhân 3 lần trên 25 câu hỏi trong tập dữ liệu. Với mỗi lần chạy, tôi sử dụng một mô hình Azure OpenAI khác nhau. Bạn có thể thấy rằng số lượng câu hỏi được trả lời đúng tăng lên khi sử dụng mô hình lớn hơn (như mong đợi). Điểm `correct_answer` được tạo bởi một [LLM-as-a-Judge Evaluator](https://langfuse.com/docs/scores/model-based-evals), được thiết lập để đánh giá mức độ chính xác của câu trả lời dựa trên câu trả lời mẫu được cung cấp trong tập dữ liệu.  

![Tổng quan về chạy tập dữ liệu](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset_runs.png)  
![So sánh chạy tập dữ liệu](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset-run-comparison.png)  



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc sự không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
